API for fetching NDFD data via their rest service. Written in python 3. In development.
Right now subGrid and getDataZipcode work well.
API Reference: https://graphical.weather.gov/xml/rest.php#degrib
Author: Tuomas Talvitie
Year: 2020

In [3]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
from urllib.parse import urlencode, quote
import xml.etree.ElementTree as ET
import xmltodict
import json

In [4]:
class ApiError(Exception):

    def __init__(self, message, status_code=None, payload=None):
        Exception.__init__(self)
        self.message = message
        if status_code is not None:
            self.status_code = status_code
        self.payload = payload
        print(self.message)
        raise Exception(self.message + ' ' + str(self.status_code))

    def to_dict(self):
        rv = dict(self.payload or ())
        rv['message'] = self.message
        print(rv['message'])
        return rv

In [6]:
#Main URL path
def _url(path=''):
    return 'http://www.weather.gov/forecasts/xml/sample_products/browser_interface/ndfdXMLclient.php?' + path


In [5]:
def subGrid(centerPointLat, centerPointLon, distanceLat, distanceLon, resolutionSquare, product, begin, end, Unit='m', optional_params=['wspd', 'wdir']):
	#Split into 3 dictionaries, each are encoded in a different manner
	params = {
		'centerPointLat':centerPointLat,
		'centerPointLon':centerPointLon,
		'distanceLat':distanceLat,
		'distanceLon':distanceLon,
		'resolutionSquare':resolutionSquare,
		'product':product
	}
	#Begin/end has special encoding
	params2 = {'begin':begin,
	'end':end
	}
	params3 = {
		'Unit':Unit,
	}

	urlString = urlencode(params)
	subString = ''
	for key, value in params2.items():
		subString += '&'+str(key) + '=' + str(value)
	urlString+=subString
	for i in optional_params:
		params3[i] = i
	urlString +='&' + urlencode(params3)

	return urlString

In [7]:
#This function acts as a general xml parser
def _generalParseXml(data):
	o = xmltodict.parse(data.content)
	d = json.dumps(o)
	d = json.loads(d)
	return d

In [8]:
def run_request(q):
	print(_url(q))
	resp = requests.get(_url(q))
	if resp.status_code != 200:
		# This means something went wrong.
		raise ApiError(resp.text, resp.status_code)
	return resp

In [9]:
def getSubGridData(centerLat, centerLon, distanceLat, distanceLon, resolutionSquare, product, begin, end, Unit='m', optional_params=[]):
	data = run_request(subGrid(centerLat, centerLon, distanceLat, distanceLon, resolutionSquare, product, begin, end, Unit='m', optional_params=['wspd', 'wdir']))
	outData = _generalParseXml(data)
	return outData

In [10]:
lat = 40.758701
lon = -111.876183
dist = 20
resolution = 20
start = '2020-07-15T00:00:00'
end = '2020-07-16T00:00:00'

In [11]:
data = getSubGridData(str(lat), str(lon), str(dist), str(dist), str(resolution), 'time-series', start, end, optional_params=['critfireo', 'dryfireo'] )

http://www.weather.gov/forecasts/xml/sample_products/browser_interface/ndfdXMLclient.php?centerPointLat=40.758701&centerPointLon=-111.876183&distanceLat=20&distanceLon=20&resolutionSquare=20&product=time-series&begin=2020-07-15T00:00:00&end=2020-07-16T00:00:00&Unit=m&wspd=wspd&wdir=wdir


In [12]:
print(data['dwml'])

{'@version': '1.0', '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xsi:noNamespaceSchemaLocation': 'https://graphical.weather.gov/xml/DWMLgen/schema/DWML.xsd', 'head': {'product': {'@srsName': 'WGS 1984', '@concise-name': 'time-series', '@operational-mode': 'official', 'title': "NOAA's National Weather Service Forecast Data", 'field': 'meteorological', 'category': 'forecast', 'creation-date': {'@refresh-frequency': 'PT1H', '#text': '2020-07-14T17:42:51Z'}}, 'source': {'more-information': 'https://graphical.weather.gov/xml/', 'production-center': {'sub-center': 'Product Generation Branch', '#text': 'Meteorological Development Laboratory'}, 'disclaimer': 'http://www.nws.noaa.gov/disclaimer.html', 'credit': 'https://www.weather.gov/', 'credit-logo': 'https://www.weather.gov/logorequest', 'feedback': 'https://www.weather.gov/contact'}}, 'data': {'location': [{'location-key': 'point1', 'point': {'@latitude': '40.53', '@longitud

In [7]:
from datetime import datetime, timedelta
start = str(datetime.now().isoformat())
end = str((datetime.now()+timedelta(days=1)).isoformat())
print(start, end)

2020-08-18T17:50:16.277146 2020-08-19T17:50:16.277205
